In [7]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit

db_path = 'd:/dw/data-warehouse/stocklab_db'
appName = "test_spark"
master = "local[*]"

conf = SparkConf().setAll([("spark.driver.maxResultSize", '6g'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client')])

spark = SparkSession.builder.appName(appName) \
  .master(master) \
  .config(conf=conf) \
  .config("spark.sql.warehouse.dir", db_path) \
  .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
  .enableHiveSupport() \
  .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

In [9]:
#spark.sql("drop database if exists stocklab CASCADE")
#AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: InvalidOperationException(message:Database stocklab is not empty. One or more tables exist.)
#force drop cascade
spark.sql("create database IF NOT EXISTS stocklab")
spark.sql("show databases ").show()
spark.sql("describe database stocklab").show(100,False)

+---------+
|namespace|
+---------+
|  default|
| stocklab|
+---------+

+-------------------------+--------------------------------------------------+
|database_description_item|database_description_value                        |
+-------------------------+--------------------------------------------------+
|Database Name            |stocklab                                          |
|Comment                  |                                                  |
|Location                 |file:/d:/dw/data-warehouse/stocklab_db/stocklab.db|
|Owner                    |Administrator                                     |
+-------------------------+--------------------------------------------------+



# min data load

df_min 으로 전체 데이터  load

In [10]:
df_min = spark.read.format("parquet").load("d:/dw/stock_min")

# dw/prod/stock_min
- load 한 데이터에 대해 shuffle data 생성
- shuffle 기준은 상위 date 기준 partition 후 각 개별 code 단위로 데이터 저장 


In [17]:
df_code = spark.read.format("parquet").load('d:/dw/stock_code')
df_code.count()

596

In [ ]:
df_min.write.format('parquet').bucketBy(20, 'shcode').mode("overwrite").saveAsTable("stocklab.stock_min_bucket")